In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import gpytorch
import matplotlib.pyplot as plt
import tqdm
import utils as utils
import scienceplots
plt.style.use('science')

base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

import fit_FaIRGP as fairgp
import fit_GP as plaingp
import evaluate_FaIRGP as evalFaIRGP
import evaluate_GP as evalGP

<IPython.core.display.Javascript object>

In [2]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126', 'ssp245', 'ssp370', 'ssp585']}}
test_data = plaingp.make_data(test_cfg)

gpmodel = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/historical-only/PlainGP/state_dict.pt')
gpmodel.load_state_dict(state_dict)

fairgpmodel = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/historical-only/FaIRGP/state_dict.pt')
fairgpmodel.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
test_scenarios = test_data.scenarios
test_times = test_scenarios.timesteps
test_tas = test_scenarios.tas
test_tas_fair = fairgpmodel._compute_mean(test_scenarios)
X_test = torch.cat([test_scenarios.cum_emissions[:, 0, None],
                    test_scenarios.emissions[:, 1:]], dim=-1)
X_test = (X_test - gpmodel.mu) / gpmodel.sigma

gpmodel = gpmodel.eval()
fairgpmodel = fairgpmodel.eval()

with torch.no_grad():
    noisy_gp_test_posterior = gpmodel.likelihood(gpmodel(X_test))
    mean = gpmodel.sigma_targets * noisy_gp_test_posterior.mean
    covar = gpmodel.sigma_targets.pow(2) * noisy_gp_test_posterior.covariance_matrix
    noisy_gp_test_posterior = noisy_gp_test_posterior.__class__(mean, covar)
    
with torch.no_grad():
    fairgp_test_posterior = fairgpmodel(test_scenarios)
    noisy_fairgp_test_posterior = fairgpmodel.likelihood(fairgp_test_posterior)
    mean = noisy_fairgp_test_posterior.mean + test_tas_fair
    covar = noisy_fairgp_test_posterior.lazy_covariance_matrix
    noisy_fairgp_test_posterior = noisy_fairgp_test_posterior.__class__(mean, covar)

/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/lazy_tensor.py:25: UserWarning: gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense
  warnings.warn("gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense")
/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/non_lazy_tensor.py:13: UserWarning: gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator
  warnings.warn("gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator")


In [5]:
gp_scores = evalGP.compute_scores(noisy_gp_test_posterior, test_scenarios)
fairgp_scores = evalFaIRGP.compute_scores(noisy_fairgp_test_posterior, test_scenarios)

In [6]:
scores_df = pd.DataFrame([fairgp_scores, gp_scores])
scores_df.columns = ['Bias', 'RMSE', 'MAE', 'Corr', 'LL', 'Calib95', 'CRPS', 'ICI']
scores_df.index = ['FaIRGP', 'Plain GP']
scores_df

,Bias,RMSE,MAE,Corr,LL,Calib95,CRPS,ICI
FaIRGP,-0.073566,0.199107,0.148063,0.976064,0.148215,0.889535,0.107950,0.025857
Plain GP,-1.689909,1.874665,1.689909,-0.363755,-30.978508,0.008721,1.558387,0.499082


In [8]:
results = scores_df[['RMSE', 'MAE', 'Bias', 'LL', 'Calib95', 'CRPS']].round(3).sort_index(ascending=False)
results.to_latex('./tables/historical-only-scores.tex')

/tmp/ipykernel_60734/749901800.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results.to_latex('./tables/historical-only-scores.tex')


In [9]:
results

,RMSE,MAE,Bias,LL,Calib95,CRPS
Plain GP,1.875,1.690,-1.690,-30.979,0.009,1.558
FaIRGP,0.199,0.148,-0.074,0.148,0.890,0.108
